<a href="https://colab.research.google.com/github/karandomguy/Phishing-Email-Detection/blob/dev/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
!pip install patool

In [27]:
import patoolib as pl
pl.extract_archive("/content/gdrive/MyDrive/phishing_url_model.zip")

patool: Extracting /content/gdrive/MyDrive/phishing_url_model.zip ...
patool: running /usr/bin/7z x -o./Unpack_frz4lm6w -- /content/gdrive/MyDrive/phishing_url_model.zip
patool: ... /content/gdrive/MyDrive/phishing_url_model.zip extracted to `best_pipeline_url.pkl'.


'best_pipeline_url.pkl'

In [28]:
%cd /content/gdrive/MyDrive/
!cp best_pipeline_tfidf.pkl /content/


/content/gdrive/MyDrive


In [29]:
import re

def check_url(text):
    url_pattern = "[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
    urls = re.findall(url_pattern, text)
    return urls

In [30]:
from urllib.parse import urlparse
from googlesearch import search

def having_ip_address(url):
    match = re.search(
        '(([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.([01]?\d\d?|2[0-4]\d|25[0-5])\.'
        '([01]?\d\d?|2[0-4]\d|25[0-5])\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\.(0x[0-9a-fA-F]{1,2})\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0

def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0

def google_index(url):
    site = search(url, 5)
    return 1 if site else 0

def count_dot(url):
    count_dot = url.count('.')
    return count_dot

def count_www(url):
    url.count('www')
    return url.count('www')

def count_atrate(url):
    return url.count('@')

def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')

def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('//')

def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0

def count_https(url):
    return url.count('https')

def count_http(url):
    return url.count('http')

def count_per(url):
    return url.count('%')

def count_ques(url):
    return url.count('?')

def count_hyphen(url):
    return url.count('-')

def count_equal(url):
    return url.count('=')

def url_length(url):
    return len(str(url))

def hostname_length(url):
    return len(urlparse(url).netloc)

def suspicious_words(url):
    match = re.search('PayPal|login|signin|bank|account|update|free|lucky|service|bonus|ebayisapi|webscr',
                      url)
    if match:
        return 1
    else:
        return 0

def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters


In [31]:
!pip install tld

In [32]:
from urllib.parse import urlparse
from tld import get_tld
import os.path

def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

def get_tld_length(url):
    try:
        tld = get_tld(url, fail_silently=True)
        return len(tld)
    except:
        return -1

In [33]:
def feature_extractor(url):
    features = []

    # Feature 1: Having IP Address
    ip_address_feature = having_ip_address(url)
    features.append(ip_address_feature)

    # Feature 2: Abnormal URL
    abnormal_url_feature=abnormal_url(url)
    features.append(abnormal_url_feature)

    # Feature 3: Count Dot
    dot_count_feature = count_dot(url)
    features.append(dot_count_feature)

    # Feature 4: Count www
    www_count_feature = count_www(url)
    features.append(www_count_feature)

    # Feature 5: Count @
    at_count_feature = count_atrate(url)
    features.append(at_count_feature)

    # Feature 6: Number of Directories
    dir_count_feature = no_of_dir(url)
    features.append(dir_count_feature)

    # Feature 7: Number of Embedded Directories
    embed_dir_count_feature = no_of_embed(url)
    features.append(embed_dir_count_feature)

    # Feature 8: Shortening Service
    shortening_service_feature = shortening_service(url)
    features.append(shortening_service_feature)

    # Feature 9: Count 'https'
    count_https_feature = count_https(url)
    features.append(count_https_feature)

    # Feature 10: Count 'http'
    count_http_feature = count_http(url)
    features.append(count_http_feature)

    # Feature 11: Count '%'
    count_per_feature = count_per(url)
    features.append(count_per_feature)

    # Feature 12: Count '?'
    count_ques_feature = count_ques(url)
    features.append(count_ques_feature)

    # Feature 13: Count '-'
    count_hyphen_feature = count_hyphen(url)
    features.append(count_hyphen_feature)

    # Feature 14: Count '='
    count_equal_feature = count_equal(url)
    features.append(count_equal_feature)

    # Feature 15: URL Length
    url_length_feature = url_length(url)
    features.append(url_length_feature)

    # Feature 16: Hostname Length
    hostname_length_feature = hostname_length(url)
    features.append(hostname_length_feature)

    # Feature 17: Suspicious Words
    suspicious_words_feature = suspicious_words(url)
    features.append(suspicious_words_feature)

    # Feature 18: Digit Count
    digit_count_feature = digit_count(url)
    features.append(digit_count_feature)

    # Feature 19: Letter Count
    letter_count_feature = letter_count(url)
    features.append(letter_count_feature)

    # Feature 20: FD Length
    fd_length_feature = fd_length(url)
    features.append(fd_length_feature)

    # Feature 21: TLD Length
    tld_length_feature = get_tld_length(url)
    features.append(tld_length_feature)

    return features

In [34]:
import joblib

def get_prediction(url, model_path):

    model = joblib.load(model_path)
    url_features = feature_extractor(url)

    print("Making prediction...")
    prediction = model.predict([url_features])
    if prediction[0]==1:
      print("Phishing")
    else:
      print("Benign")
    #print("There is", i, "% chance that this URL is malicious!")


In [35]:
model_path = r"/content/best_pipeline_url.pkl"

def url_predict(url):
  prediction = get_prediction(url,model_path)
  print(prediction)

In [36]:
import joblib

best_pipeline = joblib.load('best_pipeline_tfidf.pkl')


In [48]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
email_pipeline = joblib.load('best_pipeline_tfidf.pkl')
def preprocess(text):
      words = word_tokenize(text)
      no_stop_words = [word for word in words if word.isalpha() and word.lower() not in stop_words]
      return ' '.join(no_stop_words)

def preprocess_email(email_text):

  # Preprocess the text


  preprocessed_email = preprocess(email_text)
  preprocessed_email=preprocessed_email.lower()
  return preprocessed_email
print(stop_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'from', 'he', 'me', 'down', "doesn't", 'his', 'off', 'any', 's', 'them', 'theirs', 'over', 'she', 'these', 'here', 'doesn', "hasn't", 'they', 'am', 'been', 'itself', "couldn't", 'its', "won't", "shan't", "shouldn't", 'in', 't', "you've", 'him', 'isn', 'between', 'mustn', 'who', 'nor', 'you', 'being', 'themselves', 'once', "should've", "aren't", 'very', 'is', 'my', 'we', 'o', 'both', "it's", "didn't", "that'll", 'are', 'most', 'not', 'just', 're', 'll', 'haven', 'if', 'wouldn', 'whom', 'didn', 'there', 'have', 'before', 'too', 'into', 'will', "mustn't", 'ourselves', 'where', 'when', 'has', 'about', 'mightn', 'ma', "wouldn't", 'than', 've', "you're", 'below', 'couldn', "weren't", 'up', 'own', 'few', 'above', 'hers', 'same', 'which', 'after', "you'd", 'on', 'hasn', 'under', 'a', "don't", "haven't", 'as', 'for', 'more', 'himself', 'her', 'yourselves', 'shouldn', 'then', 'ours', 'and', 'or', 'had', 'myself', 'only', 'ain', 'should', 'did', 'do', 'against', 'yourself', 'this', 'hadn', 'can'

In [49]:
def email_predict(email_text):
  predicted_label = email_pipeline.predict([preprocess_email(email_text)])

  # Print the predicted label (1 for Safe Email, 0 for Phishing Email)
  if predicted_label[0] == '1':
    print("Predicted Label: Safe Email")
  else:
    print("Predicted Label: Phishing Email")

In [50]:
def final(text):
  urls=check_url(text)
  if len(urls)==0:
    email_predict(text)
  else:
    for i in range(len(urls)):
      url_predict(i)

In [59]:
text=input()
print(final(text))

Respected Sir,  I am writing to you on behalf of the IEEE Student Branch at the National Institute of Technology Karnataka (NITK). We are excited to announce our upcoming IEEE Week, scheduled to take place from October 1st to 5th, where we aim to foster knowledge sharing and networking among students and professionals in the field of technology.  We are particularly honored to invite you to be a distinguished speaker at our event. Given your extensive experience as a member of the IEEE Executive Committee, we believe you are uniquely positioned to address a topic of utmost importance to our audience: "Leveraging Technology for a Better Tomorrow."  We kindly request you to consider delivering a session that focuses on the positive impact of technology on society and the world at large. Your insights into how technological advancements can contribute to a more sustainable, equitable, and prosperous future will undoubtedly resonate with our students.  Your session could encompass various 

In [ ]:
f